In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import datetime

In [2]:
dataframes =pd.ExcelFile("Hari Mobile chatbot.xlsx")

In [3]:
dataframes.sheet_names

['telegram question', 'brand model', 'alternative name', 'question ', 'Sheet3']

In [4]:
telegram_question = pd.read_excel(dataframes, 'telegram question')
brand_model = pd.read_excel(dataframes, 'brand model')
alternative_name = pd.read_excel(dataframes, 'alternative name')
question = pd.read_excel(dataframes, 'question ')
sheet_3 = pd.read_excel(dataframes, 'Sheet3')

In [5]:
telegram_question.head()

,id,question,brand,Unnamed: 3,model,Unnamed: 5,Unnamed: 6,Unnamed: 7,intent 1,intent 2,Unnamed: 10
0,20026,schematic block of lenovo zuk z1,lenovo,NaN,zuk z1|z1,2,NaN,0,lock,unlock,1
1,14621,redmi 6pro ek hai local mobile shopise flash k...,redmi,NaN,zero|redmi 6|mi 6|le 2,4,NaN,0,flash,flash,1
2,13821,hi anyone with infinix zero 3 flash file i nee...,infinix,NaN,zero,1,NaN,0,flash,flash,1
3,15310,good morning anyone with infinix zero 3 flash ...,infinix,NaN,zero,1,NaN,0,flash,flash,1
4,18336,moto c plus when power button press it shows r...,moto,NaN,zero,1,NaN,0,NaN,NaN,0


In [6]:
telegram_question.loc[telegram_question['id'] == 14065]

,id,question,brand,Unnamed: 3,model,Unnamed: 5,Unnamed: 6,Unnamed: 7,intent 1,intent 2,Unnamed: 10
5680,14065,lenovo 1902 3 tested flash file pls help,lenovo,NaN,1902-03-01 00:00:00,0,NaN,0,flash,flash,1


In [7]:
telegram_question_extract = telegram_question[['brand ','model ','id']]

In [8]:
telegram_question_extract.loc[telegram_question_extract['id'] == 14065]

,brand,model,id
5680,lenovo,1902-03-01 00:00:00,14065


In [48]:
def expand_brand_models(dataframe):
    
    for index, row in dataframe.iterrows():
        if type(row['model ']) == datetime.datetime:
            dataframe.drop(index, inplace=True)
    
    brand_split = pd.concat([Series(row['model '], str(row['brand ']).split('|'))              
                    for _, row in dataframe.iterrows()]).reset_index()
    
    brand_split.rename(columns = {'index':'brand',0:'model'}, inplace = True)
    
    
    
    model_split = pd.concat([Series(row['brand'], str(row['model']).split('|'))              
                         for _, row in brand_split.iterrows()]).reset_index()
    
    model_split.rename(columns = {'index':'models',0:'brand'}, inplace = True)
    
    model_split = model_split.drop_duplicates()
    
    model_split.dropna()
    
    return model_split
    

In [49]:
models_extracted = expand_brand_models(telegram_question_extract)

In [63]:
def concat_brands_models(existing, extracted):
    all_models = pd.concat([extracted,existing])
    all_models = all_models.drop_duplicates()
    return all_models

In [55]:
models_extracted.shape

(1711, 2)

In [54]:
brand_model.shape

(7625, 2)

In [65]:
all_models = concat_brands_models(models_extracted,brand_model)
all_models.shape

/home/viky/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(9098, 2)

In [43]:
# import openpyxl
# with pd.ExcelWriter(path) as writer:
#     writer.book = openpyxl.load_workbook(path)
#     model_split.to_excel(writer, sheet_name='brand model')
#     writer.save()

In [21]:
# import pandas
# from openpyxl import load_workbook

# with pandas.ExcelWriter('temp.xlsx', engine='openpyxl') as writer:
#     writer.book = load_workbook('temp.xlsx')
#     model_split.to_excel(writer, "brand model")